In [ ]:
!pip uninstall -y ctransformers
!pip install --no-cache-dir ctransformers

In [ ]:
!mkdir -p /content/models
!ls -l /content


In [ ]:
!wget -O /content/models/openhermes-2.5-mistral-7b.Q5_K_M.gguf "https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/resolve/main/openhermes-2.5-mistral-7b.Q5_K_M.gguf"


In [ ]:
!ls -lh /content/models/
!file /content/models/openhermes-2.5-mistral-7b.Q5_K_M.gguf


In [ ]:
from ctransformers import AutoModelForCausalLM

# Define model path
model_path = "/content/models/openhermes-2.5-mistral-7b.Q5_K_M.gguf"

# Load the model
llm = AutoModelForCausalLM.from_pretrained(
    model_path,
    model_type="mistral",
    gpu_layers=100,
    context_length=4096
)

print("Model loaded successfully!")


In [ ]:
print(llm.config)

In [ ]:
# Generate a response from the model
response = llm("What are the key skills required for a software developer?")
print(response)


In [ ]:
!pip install pdfplumber chromadb sentence-transformers uvicorn ctransformers


In [ ]:
import pdfplumber
import chromadb
import uuid
from sentence_transformers import SentenceTransformer
from ctransformers import AutoModelForCausalLM


In [ ]:
def generate_response(prompt):
    return llm(prompt)

In [ ]:
# Initialize ChromaDB in-memory
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Create collection for storing resume embeddings
resume_collection = chroma_client.get_or_create_collection(name="resumes")


In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from a given PDF file dynamically."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()


In [ ]:
# Load SentenceTransformer model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")
resume_collection = chroma_client.get_or_create_collection("resume_embeddings")

def chunk_and_store_resume(resume_text):
    """Splits the extracted resume text into chunks and stores embeddings in ChromaDB."""
    chunks = resume_text.split("\n")  # Simple line-based chunking
    embeddings = embedding_model.encode(chunks).tolist()

    resume_collection.add(
        ids=[str(uuid.uuid4()) for _ in chunks],
        documents=chunks,
        embeddings=embeddings
    )


In [ ]:
def query_resume(job_role):
    """Queries the resume database for the best matching chunks based on a given job role."""
    job_embedding = embedding_model.encode([job_role]).tolist()[0]

    results = resume_collection.query(
        query_embeddings=[job_embedding],
        n_results=5
    )

    return results['documents'][0]  # Return best-matched resume chunks


In [ ]:
def analyze_resume(pdf_path, job_role):
    """Extracts text, processes the resume, and analyzes it using OpenHermes."""

    # Step 1: Extract Text
    resume_text = extract_text_from_pdf(pdf_path)

    # Step 2: Store in Vector Database
    chunk_and_store_resume(resume_text)

    # Step 3: Query Resume
    matched_text = " ".join(query_resume(job_role))

    # Step 4: Generate LLM Prompt
    prompt = f"""
    You are an excellent decision maker when it comes to select resume.
    Given the following resume details:
    {matched_text}

    And the job role: {job_role}

    Provide:
    1. **Observation** – Key strengths from the resume.
    2. **Comparison Result** – How well it matches the job role. If the candidate does not have a strong with the {job_role}, describe the gaps and suggest improvement areas to fit the role better.
    3. **Conclusion** – If it fits, motivate. If not, suggest missing skills in a list format or alternate career paths to choose with job titles.
    If you are not able to answer, respond "I dont know", do not make up an answer.
    """

    return generate_response(prompt)  # LLM model generates the response


In [ ]:
resume_file = "/content/Aditya Agarwal CV.pdf"
job_role = "Frontend Developer with React.js & Node.js experience"

analysis_result = analyze_resume(resume_file, job_role)
print(analysis_result)


In [ ]:
resume_file = "/content/Aditya Agarwal CV.pdf"
job_role = "QA Test Engineer"

analysis_result = analyze_resume(resume_file, job_role)
print(analysis_result)


In [ ]:
resume_file = "/content/Aditya Agarwal CV.pdf"
job_role = "AI Engineer"

analysis_result = analyze_resume(resume_file, job_role)
print(analysis_result)
